HW1 - Deep Learning

Requirements

In [ ]:
import numpy as np
!pip3 install numpy scipy

In [7]:
import scipy.io
import os

def load_dataset(dataset_name):
    data_path = os.path.join("./Data", f"{dataset_name}.mat")
    mat_data = scipy.io.loadmat(data_path)

    datasets = {
        'X_train': mat_data["Yt"],
        'C_train': mat_data["Ct"],
        'X_validation': mat_data["Yv"],
        'C_validation': mat_data["Cv"]
    }
    return datasets

gmm_dataset = load_dataset("GMMData")
peaks_dataset = load_dataset("PeaksData")
swiss_roll_dataset = load_dataset("SwissRollData")

In [ ]:
def soft_max_regression(x: np.ndarray):
    z = x - max(x)
    soft_max = np.exp(z) / np.sum(np.exp(z))

    return soft_max

In [ ]:
def soft_max_gradient():
    pass

In [ ]:
def gradient_test():
    pass

In [ ]:
def sgd():
    pass

In [ ]:
def sgd_with_momentum():
    pass

In [ ]:
''' BONUS PART '''


def rms_prop():
    pass